## MODULE - CASELOAD

> This code generates data for caseload analyis

> When a case spans many months, data is created for each month of it's lifespan in order to perform a variety of analysis.  In this particular scenario, we want assess the impact by month and calulate the effort required.


In [ ]:
# default_exp core.caseload

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import pandas as pd
import numpy  as np
from datetime import datetime
import altair as alt

import random
import string

#from pandas.util.testing import assert_frame_equal
from pandas._testing import assert_frame_equal
pd.set_option('display.max_columns', 30) # set so can see all columns of the DataFrame

from vega_datasets import data
_=alt.data_transformers.disable_max_rows()


CESSATION_CONTINUATION = {
    'Driver Deceased': 'Deceased', 
    'Drivers found medically unfit to drive':'Cessation',
    'Drivers that did not respond; cancelled license':'Cessation',
    'Drivers that voluntarily surrendered their license':'Cessation',
    'Drivers ultimately found fit to drive':'Continuation',
    'Cases remaining open at time of reporting':'Continuation'
}



### GET GENERATED CASELOAD DATA BY MONTH  

In [ ]:
#export
def get_generated_caseload_data_bymonth(f_path):
    filepath = f_path + 'caseloaddata_by_month.csv'
    caseload_data_by_month = pd.read_csv(filepath,parse_dates=['CASE_OPENED_DT'])
    caseload_data_by_month['Opened Month'] = caseload_data_by_month.apply(lambda x: x['CASE_OPENED_DT'].strftime('%b') + '-' + x['CASE_OPENED_DT'].strftime('%Y'), axis=1)
    
    #caseload_data_by_month['Opened Month'] = caseload_data_by_month.apply(lambda x: x['CASE_OPENED_DT'].strftime('%b') + '-' + x['CASE_OPENED_DT'].strftime('%Y'), axis=1)
    #caseload_data_by_month['Monthly Opened Count'] = caseload_data_by_month.groupby(['CASE_OPENED_DT','Year Span'])['Open Count'].transform( lambda x: sum(x))
    #caseload_data_by_month['Monthly Closed Count'] = caseload_data_by_month.groupby(['CASE_OPENED_DT','Year Span'])['Closed Count'].transform( lambda x: sum(x))

    return caseload_data_by_month



### GET PROCESSED CASE DATA

In [ ]:
#export
def get_processed_case_data(f_path):
    file_path = f_path + 'cases_processed.csv'
    cases_df = pd.read_csv(file_path,parse_dates=['BIRTHDATE','CASE_OPENED_DT','PREV_CASE_END_DT','LAST_STATUS_DATE'], dtype={'DRIVERS_LICENSE_NO': str})
    cases_df = cases_df[(cases_df['Ignore Case'] == 0) ]
    cases_df['Age Category'] = cases_df.apply( lambda x: 'Over 80 ' if x.age_bucket >= 80 else 'Under 80', axis=1)
    cases_df['Type Origin'] = cases_df.apply( lambda x: str(x['CASE_CD']) + '_' + str(x['ORIGIN_CD']), axis=1)
    cases_df['Type & Origin Desc'] = cases_df.apply( lambda x: str(x['CASE_DSC']) + ' & ' + str(x['ORIGIN_DSC']), axis=1)
    cases_df['Case Length Over 30 Days'] = cases_df.apply( lambda x: True if x['case_length_days'] >= 30 else False, axis=1)
    cases_df['Case Length Over 60 Days'] = cases_df.apply( lambda x: True if x['case_length_days'] >= 60 else False, axis=1)
    return cases_df


In [ ]:
#data_file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\\'
#generated_data_file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\\'
#cases_df = get_processed_case_data(data_file_path)

# cases_df = pd.read_csv(r'..\cases_processed.csv',parse_dates=['BIRTHDATE','CASE_OPENED_DT','PREV_CASE_END_DT','LAST_STATUS_DATE'], dtype={'DRIVERS_LICENSE_NO': str})
# cases_df = cases_df[cases_df['Ignore Case'] == 0]
# cases_df = cases_df[cases_df['Is Adjudicated'] == 'Adjudicated']
# cases_df['GENERAL_STATUS'].value_counts()

In [ ]:
#hideinput

# def f(r):
#     backlog_diff = r['Backlog Date'] - r['CASE_OPENED_DT']
#     backlog_diff = backlog_diff/np.timedelta64(1,'M')
#     last_update_diff = r['LAST_STATUS_DATE'] - r['Backlog Date'] # if same month then
#     last_update_diff = last_update_diff/np.timedelta64(1,'M')
#     if backlog_diff < 2:
#         return 0
#     elif last_update_diff <= 0 and backlog_diff < 3 :
#         return 0
#     elif ( (r['Backlog Date'].year == r['LAST_STATUS_DATE'].year ) 
#            and (r['Backlog Date'].month - r['LAST_STATUS_DATE'].month == 0 ) 
#            and ( r['GENERAL_STATUS'] == 'Closed')   
#            and backlog_diff >= 2        
#          ):
#         return -1
#     elif backlog_diff >= 2:
#         return 1
#     else:
#         return 100

# def f_isopen(r):
#     status_diff = r['LAST_STATUS_DATE'] - r['CASE_OPENED_DT']
#     status_diff = status_diff/np.timedelta64(1,'M')
#     year_span_diff = r['Year Span'] - r['LAST_STATUS_DATE'] # if same month then
#     year_span_diff = year_span_diff/np.timedelta64(1,'M')    
# #     print(diff)
# #     print(year_span_diff)
#     if year_span_diff < 1 :
#         return 'Open'
#     else:
#         return 'Closed'

    


### F CASELOAD ISOPEN

In [ ]:
#export
def f_caseload_isopen(r):
    # IS OPENED THIS MONTH
    if (r['Year Span'].month == r['CASE_OPENED_DT'].month):
        is_opened_this_month = 1
    else:
        is_opened_this_month = 0


    #check for valid 'last status date'
    testdate = pd.to_datetime(r['LAST_STATUS_DATE'], errors='ignore')

    if testdate is pd.NaT:
        status = 'Open'
        opened_count = is_opened_this_month
        closed_count = 0
        count_as_open = 1
        ignore_case = 0
        return (status, opened_count, closed_count, count_as_open, ignore_case, 99)

    
    try:
        testdate = pd.to_datetime(r['LAST_STATUS_DATE'])
    except ValueError:
        print('value error')
        status = 'Open'
        opened_count = 0
        closed_count = 0
        count_as_open = 1
        ignore_case = 0
        return (status, opened_count, closed_count, count_as_open, ignore_case, 99)
    except:
        print('generic error')
        status = 'Open'
        opened_count = 0
        closed_count = 0
        count_as_open = 1
        ignore_case = 0
        return (status, opened_count, closed_count, count_as_open, ignore_case, 100)



    # REPORT_OPEN_DIFF
    report_open_diff = r['Year Span'] - r['CASE_OPENED_DT'] # if same month then THIS IS OPENED MONTH
    report_open_diff = report_open_diff/np.timedelta64(1,'M')    
    report_open_diff = int(report_open_diff)


    # IS CLOSE MONTH
    if (r['Year Span'].month == r['LAST_STATUS_DATE'].month):
        is_close_month = 1
    else:
        is_close_month = 0
    report_status_diff = r['Year Span'] - r['LAST_STATUS_DATE'] # if same month then
    report_status_diff = report_status_diff/np.timedelta64(1,'M')   

    report_status_diff = int(report_status_diff) 

    # REPORT 
    status_diff = r['LAST_STATUS_DATE'] - r['CASE_OPENED_DT']
    status_diff = status_diff/np.timedelta64(1,'M')
    status_diff = int(status_diff)

    if 0:
        print('report_status_diff, is_close_month ', str(report_status_diff), ' ', str(is_close_month))
        print('report_open_diff, report_status_diff, statusdiff ', str(report_open_diff), ' ', str(report_status_diff), ' ',str(status_diff))
    
    if r['GENERAL_STATUS'] == 'Open':
        status = 'Open'
        # test to see if case opened this month
        if (report_open_diff == 0) :
            opened_count = 1
        else:
            opened_count = 0

        closed_count = 0
        count_as_open = 1
        ignore_case = 0

    else:
        # if ( ( ( report_open_diff <= 1) & ( report_open_diff > -1) ) & \
        #         ( ( report_status_diff < 1) & ( report_status_diff > -1) ) &  is_close_month ==1 ):  # test to see if the case both opened and closed this month:  # test to see if the case closed this month
        if  ( ( report_open_diff <= 1) & ( report_open_diff > -1) ) and \
                ( ( report_status_diff < 1) & ( report_status_diff > -1) ) and ( is_close_month ==1 ):  # test to see if the case both opened and closed this month:  # test to see if the case closed this month
            status = 'Closed'
            opened_count = is_opened_this_month
            closed_count = 1
            count_as_open = 0
            ignore_case = 0
        elif ( ( ( report_open_diff < 1) & ( report_open_diff > -1) ) and \
                ( ( report_status_diff < 1) & ( report_status_diff > -1) ) and  is_close_month == 0 ):  # test to see if the case both opened and closed this month:  # test to see if the case closed this month
            status = 'Open'
            opened_count = is_opened_this_month
            closed_count = 0
            count_as_open = 1
            ignore_case = 0

        elif ( ( report_open_diff < 1) and ( report_open_diff > -1) )  :#& ( ( year_span_diff < 1) & ( year_span_diff > -1) ):  # test to see if the case closed this month
            status = 'Open'
            opened_count = 1
            closed_count = 0
            count_as_open = 1
            ignore_case = 0
        elif report_status_diff < 0  :
            status = 'Open'
            opened_count = 1
            closed_count = 0
            count_as_open = 1
            ignore_case = 0
        elif ( ( report_status_diff < 1) and ( report_status_diff > -1) ):  # test to see if the case closed this month
            status = 'Closed'
            opened_count = 0
            closed_count = is_close_month            
            if is_close_month:
                count_as_open = 0
            else:
                count_as_open = 1
            ignore_case = 0
        else:
            status = 'Closed'
            opened_count = 0
            closed_count = 0
            count_as_open = 0
            ignore_case = 1

        
    return (status, opened_count, closed_count, count_as_open, ignore_case, report_open_diff)


In [ ]:
#export
def build_range():
    #r = pd.date_range(*(pd.to_datetime([datetime.strptime('1-1-2018', '%d-%m-%Y'), datetime.strptime('1-9-2021', '%d-%m-%Y')]) + pd.offsets.MonthEnd()), freq='M') 
    r = pd.date_range(*(pd.to_datetime([datetime.strptime('1-1-2018', '%d-%m-%Y'), datetime.strptime('1-11-2021', '%d-%m-%Y')]) + pd.offsets.MonthEnd()), freq='M') 
    return r


In [ ]:
#export
def filter_data(d, year, month):
    return d[(d.opened_year == year) & ( d.opened_month == month )].reset_index()


### BUILD MONTHLY CASELOAD

In [ ]:

#export
def build_monthly_caseload(d, end_date, filename=None):
    """given a dataframe, and an enddate for the period, return a dataframe with a case 'status info' record for each month 

    Parameters:
    dataframe (pd.DataFrame): 
    end_date (datetime): date to limit the generation of case 'status info' records
    filename (str): <todo>

    Returns:
    pd.DataFrame

   """
    #data = d[d['LAST_STATUS_DATE'].notna()].reset_index()

    data = d
    data['Year Date'] = end_date

    data['Year Span'] = [pd.date_range(*(pd.to_datetime([s, e]) + pd.offsets.MonthEnd()), freq='M') for s, e in
                  zip(pd.to_datetime(data['CASE_OPENED_DT']),
                       pd.to_datetime(data['Year Date']))]

    data = data.explode('Year Span')

    if data.empty:
        return None
    
    # changing to f_caseload to return (status, opened_count, closed_count, count_as_open, open_year_span_diff)

    data[['Status', 'Opened Count', 'Closed Count', 'Count as Open', 'Ignore Case',  'open_year_span_diff']] = data.apply( f_caseload_isopen   , axis=1).to_list()

    if filename:
        summaryfilename = filename.replace('.csv', '_Adjudicated_Summary.csv' ) #Jan_2018_toSept2021.csv

        print(filename)
        #data.to_csv(filename, index = False)

    data = data[data['Ignore Case'] == 0].reset_index(drop=True)

    return data



### AGGREGATE MONTHLY DATA

In [ ]:

#export
def aggregate_monthly_data(data, ag_dict={}):
    """given a dataframe, and an enddate for the period, return a dataframe with a case 'status info' record for each month 

    Parameters:
    data (pd.DataFrame): 
    by (list): list of fields to group by

    Returns:
    pd.DataFrame

    """
    # we have determined impact for each case, now aggregate numbers so we have the impact for the month of data
    if len(ag_dict.keys()) > 0:
        aggregation = ag_dict
    else:
        aggregation = {
        'Opened Count': ('Opened Count','sum'),
        'Count as Open': ('Count as Open','sum'),
        'Closed': ('Closed Count','sum')
        # 'Case Count': ('DRIVERS_LICENSE_NO','count'),
        #     'Status Count': ('STATUS_COUNT', 'sum')
        }

    monthly_counts = data.groupby([pd.Grouper(freq='M', key='CASE_OPENED_DT') , 
                                  pd.Grouper(key='Year Span'),
                                  pd.Grouper(key='Status'),
                                  pd.Grouper(key='Age Category'),                                   
                                 ]).agg(** aggregation)
    
    monthly_counts = pd.DataFrame(monthly_counts).reset_index()
    #monthly_counts.to_csv(summaryfilename, index = False)
    
    return monthly_counts

### GENERATE CASELOAD DATA


In [ ]:
#export
def generate_caseload_data(dataframe):
    """process the case data month-by=month

    Parameters:
    dataframe (pd.DataFrame)
    f_path (str): the file path wher the generated caseload data should be put

    Returns:
    pd.DataFrame

    """

    caseload_data = pd.DataFrame()

    r = build_range()
    #for date in r[:2]:
    for date in r:

        if (date.year == r[-1].year) and (date.month == r[-1].month) : break

        #filename = generated_data_file_path + date.strftime("%b") + '_' + date.strftime("%Y") + '_' + 'toNov2021_ADJUDICATEDCASELOAD' + '.csv' 

        monthly_data = filter_data(dataframe, date.year, date.month)    

        caseload_month = build_monthly_caseload(monthly_data,r[-1])  # call func without file path so the intermediate files don't get writtn
        
        #caseload_month = build_monthly_caseload(monthly_data,r[-1], filename)
        
        if caseload_month is None: 
            break
        if caseload_data.empty:
            caseload_data = caseload_month
        else:
            caseload_data = caseload_data.append(caseload_month)

    return caseload_data



### GENERATE AND WRITE CASELOAD DATA

In [ ]:
#export
def generate_and_write_caseload_data(dataframe, f_path, f_name=None):
    """process the case data month-by=month

    Parameters:
    dataframe (pd.DataFrame)
    f_path (str): the file path wher the generated caseload data should be put

    Returns:
    pd.DataFrame

    """
    if f_name is None:
        filename = f_path + 'caseloaddata_by_month.csv'
    else:
        filename = f_path + f_name
    caseload_data = generate_caseload_data(dataframe)
    caseload_summary = aggregate_monthly_data(caseload_data)
    caseload_summary.to_csv(filename, index = False)
    return caseload_summary
    # caseload_data.to_csv(filename, index = False)
    # return caseload_data


In [ ]:
if 0:
    caseload_data = generate_caseload_data()
    caseload_data['Cases_cumsum'] = caseload_data.groupby(['CASE_OPENED_DT','Year Span'])['Case Count'].cumsum()
else:
    caseload_data = False

In [ ]:
if caseload_data:

    aggregation = {
         'Open Count': ('Open Count','sum')
    }
    jan_data = caseload_data[caseload_data['CASE_OPENED_DT'] == '2018-01-31']

    backlog_data_byday = jan_data.groupby(['Year Span']) \
    .agg(** aggregation).reset_index()

else:
    backlog_data_byday = 'Nothing to be done'
backlog_data_byday

'Nothing to be done'

In [ ]:
if  backlog_data_byday != 'Nothing to be done':
    montly_backlog_all = alt.Chart(backlog_data_byday).mark_area(point=True).encode(
        y = alt.Y("Open Count:Q" ),
        x = alt.X("Year Span:T",  scale=alt.Scale(zero=False) ),
    #   color = "Year Span:N",
    #    tooltip=['Backlog Total']
    ).properties(
        width=700,
        height=400
    )
else:
    montly_backlog_all = backlog_data_byday

montly_backlog_all

'Nothing to be done'

In [ ]:
backlog_data_byday

'Nothing to be done'

In [ ]:
if  backlog_data_byday != 'Nothing to be done':
    aggregation = {
         'Open Count': ('Open Count','sum')
    #     'Cases': ('Cases','sum'),
    }

    backlog_data_byday = jan_data.groupby(['Year Span', 'Age Category']) \
    .agg(** aggregation).reset_index()

    montly_backlog_all = alt.Chart(backlog_data_byday).mark_area(point=True).encode(
        y = alt.Y("sum(Open Count):Q" ),
        x = alt.X("Year Span:T",  scale=alt.Scale(zero=False) ),
       color = "Age Category:N",
       tooltip=['sum(Open Count)']
    ).properties(
        width=700,
        height=400
    )
else:
    montly_backlog_all = backlog_data_byday

montly_backlog_all


'Nothing to be done'

In [ ]:
if  backlog_data_byday != 'Nothing to be done':
    aggregation = {
         'Open Count': ('Open Count','sum'),
       'Total Cases': ('Case Count','sum'),
    }

    backlog_data_byday = jan_data.groupby(['Year Span', 'Age Category','CASE_OPENED_DT']) \
    .agg(** aggregation).reset_index()

    montly_backlog_all = alt.Chart(backlog_data_byday).mark_area(point=True).encode(
        y = alt.Y("sum(Open Count):Q" ),
        x = alt.X("Year Span:T",  scale=alt.Scale(zero=False) ),
       color = "CASE_OPENED_DT:N",
        tooltip=['CASE_OPENED_DT:T']
    ).properties(
        width=700,
        height=400
    )
else:
    montly_backlog_all = backlog_data_byday

montly_backlog_all



'Nothing to be done'

In [ ]:
if caseload_data:
    filepath = generated_data_file_path + 'caseloaddata_by_month.csv' 
    caseload_data.to_csv(filepath, index = False)
else:
    print(montly_backlog_all)

Nothing to be done


## SAMPLE DATA

> We need to prepare a set of data to send to the function that explodes the cases into records for each month in > it's lifecycle


Our SAMPLE dataframe needs the following columns:

* CASE_OPENED_DT
* LAST_STATUS_DATE
* DRIVERS_LICENSE_NO
* STATUS_COUNT
* Year Span
* Status : 'Open' or 'Closed'
* Age Category: 'Over 80' or 'Under 80'

columns = ['CASE_OPENED_DT','LAST_STATUS_DATE','DRIVERS_LICENSE_NO','STATUS_COUNT','Year Span','Status','Age Category']

                        


#### TEMP CODE DOC 

 from build_monthly_caseload

def build_monthly_caseload(d, end_date, filename):
    summaryfilename = filename.replace('.csv', '_Adjudicated_Summary.csv' ) #Jan_2018_toSept2021.csv
    data = d[d['LAST_STATUS_DATE'].notna()].reset_index()
    data['Year Date'] = end_date

    data['Year Span'] = [pd.date_range(*(pd.to_datetime([s, e]) + pd.offsets.MonthEnd()), freq='M') for s, e in
                  zip(pd.to_datetime(data['CASE_OPENED_DT']),
                       pd.to_datetime(data['Year Date']))]

    data = data.explode('Year Span')

    if data.empty:
        return None
    
    data[['Status', 'Open Count', 'Closed Count']] = data.apply( f_caseload_isopen   , axis=1).to_list()
    data.to_csv(filename, index = False)
    
    # we have determined impact for each case, now aggregate numbers so we have the impact for the month of data


    aggregation = {
    'Open Count': ('Open Count','sum'),
    'Closed Count': ('Closed Count','sum'),
    'Case Count': ('DRIVERS_LICENSE_NO','count'),
        'Status Count': ('STATUS_COUNT', 'sum')
    }

    monthly_counts = data.groupby([pd.Grouper(freq='M', key='CASE_OPENED_DT') , 
                                  pd.Grouper(key='Year Span'),
                                  pd.Grouper(key='Status'),
                                  pd.Grouper(key='Age Category'),                                   
                                 ]).agg(** aggregation)
    
    monthly_counts = pd.DataFrame(monthly_counts).reset_index()
    monthly_counts.to_csv(summaryfilename, index = False)
    
    return monthly_counts





#### TEMP CODE DOC

```

def build_monthly_caseload(d, end_date, filename):
    summaryfilename = filename.replace('.csv', '_Adjudicated_Summary.csv' ) #Jan_2018_toSept2021.csv
    data = d[d['LAST_STATUS_DATE'].notna()].reset_index()
    data['Year Date'] = end_date

    data['Year Span'] = [pd.date_range(*(pd.to_datetime([s, e]) + pd.offsets.MonthEnd()), freq='M') for s, e in
                  zip(pd.to_datetime(data['CASE_OPENED_DT']),
                       pd.to_datetime(data['Year Date']))]

    data = data.explode('Year Span')

    if data.empty:
        return None
    
    data[['Status', 'Open Count', 'Closed Count']] = data.apply( f_caseload_isopen   , axis=1).to_list()
    data.to_csv(filename, index = False)
    
    # we have determined impact for each case, now aggregate numbers so we have the impact for the month of data


    aggregation = {
    'Open Count': ('Open Count','sum'),
    'Closed Count': ('Closed Count','sum'),
    'Case Count': ('DRIVERS_LICENSE_NO','count'),
        'Status Count': ('STATUS_COUNT', 'sum')
    }

    monthly_counts = data.groupby([pd.Grouper(freq='M', key='CASE_OPENED_DT') , 
                                  pd.Grouper(key='Year Span'),
                                  pd.Grouper(key='Status'),
                                  pd.Grouper(key='Age Category'),                                   
                                 ]).agg(** aggregation)
    
    monthly_counts = pd.DataFrame(monthly_counts).reset_index()
    monthly_counts.to_csv(summaryfilename, index = False)
    
    return monthly_counts
```



In [ ]:
opened_dt = [
    '2018-09-13'  
    ,'2018-09-13'
    ,'2018-09-23'
    ,'2018-09-23'
    ,'2018-09-23'
    ,'2018-09-23'
    ,'2018-09-23'
    ,'2018-10-13'
    ,'2018-10-23'
    ,'2018-10-23' ]
last_status_dt = [
    '2018-09-13'  
    ,'2018-09-23'
    ,'2018-10-23'
    ,'2018-11-23'
    ,'2018-11-23'
    ,'2018-11-23'
    ,'2018-11-23'
    ,'2018-10-13'
    ,'2018-11-23'    
    ,'2018-12-23' ]

dl_no = [
    '990308'
,    '191228'
,    '184859'
,    '889368'
,    '536602'
,    '190550'
,    '022627'
,    '769820'
,    '304546'
,    '308591'
]

scenario_type = [
    'BASIC - SAME MONTH'
,    'BASIC - SAME MONTH'
,    'BASIC - 1 MONTH'
,    'BASIC - 3 MONTHS'
,    ''
,    ''
,    ''
,    ''
,    ''
,    'STILL OPEN'
]
    
#output_string = ''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(10))

zipped = list(zip( scenario_type, pd.to_datetime(opened_dt), pd.to_datetime(last_status_dt), dl_no ))

sample_cases_df = pd.DataFrame(zipped, columns=['SCENARIO', 'CASE_OPENED_DT','LAST_STATUS_DATE', 'DRIVERS_LICENSE_NO'])

#cases_df['DRIVERS_LICENSE_NO'] = cases_df.apply( lambda x: ''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(6)), axis=1)
#    cases_df['DRIVERS_LICENSE_NO'] = cases_df.apply( lambda x: ''.join(random.SystemRandom().choice(string.digits) for _ in range(6)), axis=1)
sample_cases_df['STATUS_COUNT'] = sample_cases_df.apply( lambda x: 5, axis=1)
sample_cases_df['Age Category'] = sample_cases_df.apply( lambda x: 'Over 80', axis=1)
sample_cases_df['GENERAL_STATUS'] = sample_cases_df.apply( lambda x: 'Closed', axis=1)
sample_cases_df.iloc[9,6] = 'Open'
sample_cases_df

    

,SCENARIO,CASE_OPENED_DT,LAST_STATUS_DATE,DRIVERS_LICENSE_NO,STATUS_COUNT,Age Category,GENERAL_STATUS
0,BASIC - SAME MONTH,2018-09-13,2018-09-13,990308,5,Over 80,Closed
1,BASIC - SAME MONTH,2018-09-13,2018-09-23,191228,5,Over 80,Closed
2,BASIC - 1 MONTH,2018-09-23,2018-10-23,184859,5,Over 80,Closed
3,BASIC - 3 MONTHS,2018-09-23,2018-11-23,889368,5,Over 80,Closed
4,,2018-09-23,2018-11-23,536602,5,Over 80,Closed
5,,2018-09-23,2018-11-23,190550,5,Over 80,Closed
6,,2018-09-23,2018-11-23,022627,5,Over 80,Closed
7,,2018-10-13,2018-10-13,769820,5,Over 80,Closed
8,,2018-10-23,2018-11-23,304546,5,Over 80,Closed
9,STILL OPEN,2018-10-23,2018-12-23,308591,5,Over 80,Open


### Close Analysis BASIC 1 MONTH EXACTLY


In [ ]:

opened_date = ['2018-10-23', '2018-10-23', '2018-10-23'] #CASE_OPENED_DT	
opened_count = [1, 0] #Opened Count	
count_as_open = [1, 0] #Count as Open	
closed_count = [0,1] #Closed Count	
ignore_case = [0,0] #Ignore Case

zipped = list(zip(  opened_count, count_as_open, closed_count, ignore_case ))
right_side_test = pd.DataFrame(zipped, columns=['Opened Count', 'Count as Open', 'Closed Count', 'Ignore Case'])
#print(right_side_test)





driver_184859 =  sample_cases_df[sample_cases_df['DRIVERS_LICENSE_NO'] == '184859'].reset_index(drop=True)

year_span_dt = pd.to_datetime('20190101', format='%Y%m%d', errors='ignore')

caseload_184859_month = build_monthly_caseload(driver_184859, year_span_dt)
left_side = caseload_184859_month[['Opened Count', 'Count as Open', 'Closed Count', 'Ignore Case']]

caseload_184859_month[['CASE_OPENED_DT', 'Status', 'Opened Count', 'Count as Open',  'Closed Count', 'Ignore Case']]



,CASE_OPENED_DT,Status,Opened Count,Count as Open,Closed Count,Ignore Case
0,2018-09-23,Open,1,1,0,0
1,2018-09-23,Closed,0,0,1,0


#### TEST => BASIC 1 MONTH 

### Close analysis BASIC 3 MONTHS

In [ ]:
print(left_side)
print(right_side_test)


assert_frame_equal(left_side, right_side_test, check_dtype=False)


  Opened Count Count as Open Closed Count Ignore Case
0            1             1            0           0
1            0             0            1           0
   Opened Count  Count as Open  Closed Count  Ignore Case
0             1              1             0            0
1             0              0             1            0


In [ ]:

year_span_dt = pd.to_datetime('20190101', format='%Y%m%d', errors='ignore')

driver_889368 =  sample_cases_df[sample_cases_df['DRIVERS_LICENSE_NO'] == '889368'].reset_index(drop=True)
caseload_889368_month = build_monthly_caseload(driver_889368, year_span_dt)

caseload_889368_month[['CASE_OPENED_DT','Opened Count', 'Count as Open',  'Closed Count', 'Ignore Case']]



,CASE_OPENED_DT,Opened Count,Count as Open,Closed Count,Ignore Case
0,2018-09-23,1,1,0,0
1,2018-09-23,0,1,0,0
2,2018-09-23,0,0,1,0


### Close analysis STILL OPEN

There is logic build around the last status data

In [ ]:

driver_308591 =  sample_cases_df[sample_cases_df['DRIVERS_LICENSE_NO'] == '308591'].reset_index(drop=True)
caseload_308591_month = build_monthly_caseload(driver_308591, year_span_dt)
#caseload_308591_month
caseload_308591_month[['CASE_OPENED_DT','Opened Count', 'Count as Open',  'Closed Count', 'Ignore Case']]


,CASE_OPENED_DT,Opened Count,Count as Open,Closed Count,Ignore Case
0,2018-10-23,1,1,0,0
1,2018-10-23,0,1,0,0
2,2018-10-23,0,1,0,0
3,2018-10-23,0,1,0,0


In [ ]:

#caseload_month = *build_monthly_caseload*(monthly_data,r[-1], filename)

#pd.to_datetime('13000101', format='%Y%m%d', errors='ignore')
year_span_dt = pd.to_datetime('20190101', format='%Y%m%d', errors='ignore')

caseload_month = build_monthly_caseload(sample_cases_df, year_span_dt, 'bla')


bla


In [ ]:
caseload_month = caseload_month[caseload_month['Ignore Case'] == 0]


alt.Chart(caseload_month).mark_bar().encode(
    y='DRIVERS_LICENSE_NO',
    x='count(DRIVERS_LICENSE_NO):Q',
#    y='STATUS_COUNT:Q',
    color='Status'
)

alt.Chart(...)

In [ ]:
#summary = caseload_month.groupby(['Year Span'])['Open Count'].sum()

aggregation = {
         'Newly Opened Count': ('Opened Count','sum'),
          'Newly Closed Cases': ('Closed Count','sum'),
         'Currently Opened Count': ('Count as Open','sum')
         #, 'Ignored Count': ('Ignore Case','sum')
       #,'Activity Count': ('DRIVERS_LICENSE_NO','count')
    }

summary = caseload_month.groupby(['Year Span']) \
.agg(** aggregation).reset_index()

summary_df = pd.DataFrame(summary).reset_index(drop=True)
#summary_df['Cumulative Total'] = summary_df['']
summary_df


,Year Span,Newly Opened Count,Newly Closed Cases,Currently Opened Count
0,2018-09-30,7,2,5
1,2018-10-31,3,2,6
2,2018-11-30,0,5,1
3,2018-12-31,0,0,1
4,2019-01-31,0,0,1


#### TEST => Summary Data
But what we really want is the following

In [ ]:
newly_opened_count = [7,3,0,0,0] #Newly Opened Count
newly_closed_cases = [2, 2,5,0,0] #Newly Closed Cases	
currently_open = [5, 6, 1, 1, 1] #Currently Opened Count

zipped = list(zip(  newly_opened_count, newly_closed_cases, currently_open ))
right_side_test = pd.DataFrame(zipped, columns=['Newly Opened Count', 'Newly Closed Cases', 'Currently Opened Count'])


#test_df.iloc[2,2] = 1

left_side = summary_df[['Newly Opened Count', 'Newly Closed Cases', 'Currently Opened Count']].reset_index(drop=True)

assert_frame_equal(left_side, right_side_test, check_dtype=False)



#test_df.iloc[3,2] = 0
#test_df.iloc[3,3] = 0

#test_df.iloc[2,3] = 2




### Build Monthly Caseload => TEST

> build_monthly_caseload 

In [ ]:

year_span_dt = pd.to_datetime('20190101', format='%Y%m%d', errors='ignore')

left_side_test = build_monthly_caseload(sample_cases_df, year_span_dt)

summary_data = aggregate_monthly_data(left_side_test)

summary_data

,CASE_OPENED_DT,Year Span,Status,Age Category,Opened Count,Count as Open,Closed
0,2018-09-30,2018-09-30,Closed,Over 80,2,0,2
1,2018-09-30,2018-09-30,Open,Over 80,5,5,0
2,2018-09-30,2018-10-31,Closed,Over 80,0,4,1
3,2018-09-30,2018-11-30,Closed,Over 80,0,0,4
4,2018-10-31,2018-10-31,Closed,Over 80,1,0,1
5,2018-10-31,2018-10-31,Open,Over 80,2,2,0
6,2018-10-31,2018-11-30,Closed,Over 80,0,0,1
7,2018-10-31,2018-11-30,Open,Over 80,0,1,0
8,2018-10-31,2018-12-31,Open,Over 80,0,1,0
9,2018-10-31,2019-01-31,Open,Over 80,0,1,0


In [ ]:
left_side = caseload_889368_month[['CASE_OPENED_DT','Opened Count', 'Count as Open', 'Closed Count', 'Ignore Case']]

left_side

,CASE_OPENED_DT,Opened Count,Count as Open,Closed Count,Ignore Case
0,2018-09-23,1,1,0,0
1,2018-09-23,0,1,0,0
2,2018-09-23,0,0,1,0


In [ ]:
#right_side 

opened_date = ['2018-09-23', '2018-09-23', '2018-09-23', '2018-09-23'] #CASE_OPENED_DT	
opened_count = [1, 0, 0] #Opened Count	
count_as_open = [1, 1, 0] #Count as Open	
closed_count = [0,0,1] #Closed Count	
ignore_case = [0,0,0] #Ignore Case

zipped = list(zip( pd.to_datetime(opened_date), opened_count, count_as_open, closed_count, ignore_case ))
right_side_test = pd.DataFrame(zipped, columns=['CASE_OPENED_DT','Opened Count', 'Count as Open', 'Closed Count', 'Ignore Case'])
#right_side_test

assert_frame_equal(left_side, right_side_test, check_dtype=False)
    

### F Caseload Is Open => TESTS

In [ ]:
#pd.Timestamp.


In [ ]:

# driver_308591 =  sample_cases_df[sample_cases_df['DRIVERS_LICENSE_NO'] == '308591'].reset_index(drop=True)
# caseload_308591_month = build_monthly_caseload(driver_308591, year_span_dt)
# #caseload_308591_month
# caseload_308591_month[['CASE_OPENED_DT','Opened Count', 'Count as Open',  'Closed Count', 'Ignore Case']]

#sample_cases_df[['Status', 'Opened Count', 'Closed Count', 'Count as Open', 'Ignore Case',  'open_year_span_diff']] = sample_cases_df.apply( f_caseload_isopen   , axis=1).to_list()

year_span_dt = pd.to_datetime('20190101', format='%Y%m%d', errors='ignore')

left_side_test = build_monthly_caseload(sample_cases_df, year_span_dt)

left_side_test[['Status', 'Opened Count', 'Closed Count', 'Count as Open', 'Ignore Case',  'open_year_span_diff']] = left_side_test.apply( f_caseload_isopen   , axis=1).to_list()

left_side_test.head()

,SCENARIO,CASE_OPENED_DT,LAST_STATUS_DATE,DRIVERS_LICENSE_NO,STATUS_COUNT,Age Category,GENERAL_STATUS,Year Date,Year Span,Status,Opened Count,Closed Count,Count as Open,Ignore Case,open_year_span_diff
0,BASIC - SAME MONTH,2018-09-13,2018-09-13,990308,5,Over 80,Closed,2019-01-01,2018-09-30,Closed,1,1,0,0,0
1,BASIC - SAME MONTH,2018-09-13,2018-09-23,191228,5,Over 80,Closed,2019-01-01,2018-09-30,Closed,1,1,0,0,0
2,BASIC - 1 MONTH,2018-09-23,2018-10-23,184859,5,Over 80,Closed,2019-01-01,2018-09-30,Open,1,0,1,0,0
3,BASIC - 1 MONTH,2018-09-23,2018-10-23,184859,5,Over 80,Closed,2019-01-01,2018-10-31,Closed,0,1,0,0,1
4,BASIC - 3 MONTHS,2018-09-23,2018-11-23,889368,5,Over 80,Closed,2019-01-01,2018-09-30,Open,1,0,1,0,0


In [ ]:

driver_889368 =  sample_cases_df[sample_cases_df['DRIVERS_LICENSE_NO'] == '889368'].reset_index(drop=True)
driver_889368.iloc[0,2] = ''

# caseload_308591_month = build_monthly_caseload(driver_308591, year_span_dt)
# #caseload_308591_month
# caseload_308591_month[['CASE_OPENED_DT','Opened Count', 'Count as Open',  'Closed Count', 'Ignore Case']]

#sample_cases_df[['Status', 'Opened Count', 'Closed Count', 'Count as Open', 'Ignore Case',  'open_year_span_diff']] = sample_cases_df.apply( f_caseload_isopen   , axis=1).to_list()



year_span_dt = pd.to_datetime('20190101', format='%Y%m%d', errors='ignore')

left_side_test = build_monthly_caseload(driver_889368, year_span_dt)

left_side_test[['Status', 'Opened Count', 'Closed Count', 'Count as Open', 'Ignore Case',  'open_year_span_diff']] = left_side_test.apply( f_caseload_isopen   , axis=1).to_list()

left_side_test

,SCENARIO,CASE_OPENED_DT,LAST_STATUS_DATE,DRIVERS_LICENSE_NO,STATUS_COUNT,Age Category,GENERAL_STATUS,Year Date,Year Span,Status,Opened Count,Closed Count,Count as Open,Ignore Case,open_year_span_diff
0,BASIC - 3 MONTHS,2018-09-23,NaT,889368,5,Over 80,Closed,2019-01-01,2018-09-30,Open,1,0,1,0,99
1,BASIC - 3 MONTHS,2018-09-23,NaT,889368,5,Over 80,Closed,2019-01-01,2018-10-31,Open,0,0,1,0,99
2,BASIC - 3 MONTHS,2018-09-23,NaT,889368,5,Over 80,Closed,2019-01-01,2018-11-30,Open,0,0,1,0,99
3,BASIC - 3 MONTHS,2018-09-23,NaT,889368,5,Over 80,Closed,2019-01-01,2018-12-31,Open,0,0,1,0,99
4,BASIC - 3 MONTHS,2018-09-23,NaT,889368,5,Over 80,Closed,2019-01-01,2019-01-31,Open,0,0,1,0,99


### Get Generated Case Data by Month ==> TEST

In [ ]:
#hide
generateddata_file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\Generated\\'

df = get_generated_caseload_data_bymonth(generateddata_file_path)
df

,CASE_OPENED_DT,Year Span,Status,Age Category,Opened Count,Count as Open,Closed,Opened Month
0,2018-01-31,2018-01-31,Closed,Over 80,409,0,409,Jan-2018
1,2018-01-31,2018-01-31,Closed,Under 80,1116,0,1116,Jan-2018
2,2018-01-31,2018-01-31,Open,Over 80,3700,3700,0,Jan-2018
3,2018-01-31,2018-01-31,Open,Under 80,7667,7667,0,Jan-2018
4,2018-01-31,2018-02-28,Closed,Over 80,0,1015,1778,Jan-2018
...,...,...,...,...,...,...,...,...
1791,2018-12-31,2021-10-31,Open,Under 80,0,8,0,Dec-2018
1792,2018-12-31,2021-11-30,Open,Over 80,0,1,0,Dec-2018
1793,2018-12-31,2021-11-30,Open,Under 80,0,8,0,Dec-2018
1794,2018-12-31,2021-12-31,Open,Over 80,0,1,0,Dec-2018


In [ ]:
# pivot = summary_df.pivot_table( values = 'Open Count', columns = 'Year Span').reset_index()
# pivot.reset_index(drop=True)

# aggregation = {
#          'Open Count': ('Open Count','sum'),
#        'Closed Cases': ('Closed Count','sum'),
#        'Case Count': ('Closed Count','sum')
#     }

# backlog_data_byday = jan_data.groupby(['Year Span']) \
# .agg(** aggregation).reset_index()



In [ ]:
open = alt.Chart(caseload_month).mark_line().encode(
    y='sum(Opened Count)',
    x='Year Span:T',
#    y='STATUS_COUNT:Q',
#    color='Status'
)

open | open.encode( y='sum(Closed Count):Q') | open.encode( y='sum(Count as Open):Q')

alt.HConcatChart(...)

In [ ]:
#hide

from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_core.baseline.ipynb.
Converted 02_core.caseload.ipynb.
Converted index.ipynb.


In [ ]:
# from nbdev.test import test_nb
# test_nb('02_core.caseload.ipynb')
